In [1]:
import ibm_db
import ibm_db_dbi
import ibm_db_sa
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime as dt
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

# Kết nối với DB2
dsn = "DRIVER={IBM DB2 ODBC DRIVER};DATABASE=bludb;HOSTNAME=10.1.50.25;PORT=50000;PROTOCOL=TCPIP;UID=tpbdatait1;PWD=Tpbdatait1@123;"
hdbc = ibm_db.connect(dsn, "", "")
hdbi = ibm_db_dbi.Connection(hdbc)

# Display option
pd.options.display.float_format = '{:.4f}'.format
np.set_printoptions(suppress= True)

# Tao danh sach Mahalanobis

In [2]:
ewallet = pd.read_csv(wslib.mount.get_data_path('twt_ebank_ewallet.csv'), index_col=0)
ewallet['NGAY_GD'] = pd.to_datetime(ewallet['NGAY_GD'])
ewallet['SO_CIF'] = ewallet['SO_CIF'].astype('str').str.zfill(8)
ewallet.rename(columns= {'SO_CIF':'CIF'}, inplace= True)
ewallet.head()

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,LOAI_GD,NGAY_GD,KENH_GIAO_DICH,LOAI_VI,CIF,STATUS,SO_VI,AMOUNT,MA_GIAO_DICH,TRACENUMBER
0,Vi dien tu,2021-09-14,CONFIRM NAP TIEN TU VI,MOMO,00325849,Thanh cong,972152262.0000,88000.0000,384334498,16577426443
1,Vi dien tu,2021-09-14,CONFIRM NAP TIEN TU VI,MOMO,03859507,Thanh cong,362248159.0000,3343000.0000,384346323,16577919174
2,Vi dien tu,2021-09-14,CONFIRM NAP TIEN EBANKX,ESB2,09361631,Thanh cong,NaN,682000.0000,384349046,HYD3265583921377824
3,Vi dien tu,2021-09-14,CONFIRM NAP TIEN TU VI,AIRPAY,01904282,Thanh cong,917001979.0000,173500.0000,384351931,2109140300322529
4,Vi dien tu,2021-09-14,CONFIRM NAP TIEN TU VI,MOMO,03357289,Thanh cong,354657793.0000,12000.0000,384357826,16578251428


In [5]:
# Chon thoi diem look back
dfe = ewallet.loc[(ewallet['NGAY_GD'] <= pd.to_datetime('2021-11-30'))]

# Tinh toan cac gia tri
count = dfe.groupby('CIF',as_index=False)['TRACENUMBER'].count()

mean = dfe.groupby('CIF',as_index=False)['AMOUNT'].mean()
mean.rename(columns= {'AMOUNT':'Mean'}, inplace= True)

std = dfe.groupby('CIF',as_index=False)['AMOUNT'].std()

# Tao bang ket qua
data = count.merge(mean, on = 'CIF', how = 'outer').merge(std, on= 'CIF', how = 'outer')
data.rename(columns= {'TRACENUMBER':'Count', 'AMOUNT':'Std'}, inplace= True)
# data = data[data['Count'] >5]
data.head()

,CIF,Count,Mean,Std
0,+8496222,1,50000.0000,NaN
1,+8498999,1,20000.0000,NaN
2,00000000,1,10000.0000,NaN
3,00000008,12,541707.1667,301812.4489
4,00000066,1,100000.0000,NaN


In [4]:
# start_date = pd.to_datetime('2020-01-01')
# end_date = pd.to_datetime('2021-12-01')
# sample_data = ewallet.loc[(ewallet['DATE'] >= start_date) & (ewallet['DATE'] < end_date)]

In [6]:
# Chon thoi diem quan sat
range_min = pd.to_datetime('2021-12-01')
range_max = pd.to_datetime('2021-12-31')

# Tinh khoang cach mahalanobis
max_data = pd.DataFrame()
for i in pd.date_range(range_min,range_max):
    df = ewallet[ewallet['NGAY_GD'] ==i]
    temp = df.groupby(['NGAY_GD', 'CIF'], as_index= False)['AMOUNT'].max()
    max_data = max_data.append(temp)
max_data.rename(columns= {'AMOUNT': 'Max_amt'}, inplace= True)
mahala_data = max_data.merge(data, on= 'CIF', how= 'inner')
mahala_data['Mahala_distance'] = (mahala_data['Max_amt'] - mahala_data['Mean'])/ mahala_data['Std']
mahala_data.head()

,NGAY_GD,CIF,Max_amt,Count,Mean,Std,Mahala_distance
0,2021-12-01,00000113,300000.0000,50,242840.1600,342024.4388,0.1671
1,2021-12-01,00000177,145600.0000,40,70495.1500,54915.3627,1.3676
2,2021-12-01,00000399,40000.0000,213,180635.5822,235857.5755,-0.5963
3,2021-12-03,00000399,45000.0000,213,180635.5822,235857.5755,-0.5751
4,2021-12-04,00000399,56500.0000,213,180635.5822,235857.5755,-0.5263


In [7]:
# Drop mahalanobis_distance = inf
mahala_drop0 = mahala_data[mahala_data['Std'] != 0]

#Tao list trong
list_i = []
list_cus = []
list_cover = []
list_maha_mean = []
list_maha_std = []

# Tinh ratio
for i in range(1, 13):
# Tinh cac ty le
    customer = data[data['Count']>=i]['CIF'].shape[0]
    cover_rate = customer/data.shape[0]
    df = mahala_drop0[mahala_drop0['Count'] >=i]
    mahala_mean = df['Mahala_distance'].mean()
    mahala_std = df['Mahala_distance'].std()
# Append list    
    list_i.append(i)
    list_cus.append(customer)
    list_cover.append(cover_rate)
    list_maha_mean.append(mahala_mean)
    list_maha_std.append(mahala_std)
ratio = pd.DataFrame({'So GD toi thieu': list_i
                      , 'So KH': list_cus
                      , 'Ty le cover': list_cover
                     , 'Mahala_mean': list_maha_mean
                     , 'Mahala_std': list_maha_std})
ratio

,So GD toi thieu,So KH,Ty le cover,Mahala_mean,Mahala_std
0,1,496680,1.0000,0.6053,39.2771
1,2,399136,0.8036,0.6053,39.2771
2,3,360218,0.7253,0.4964,5.8821
3,4,336446,0.6774,0.4688,4.9333
4,5,319337,0.6429,0.4536,3.6193
5,6,305719,0.6155,0.4433,3.4887
6,7,294525,0.5930,0.4372,3.4226
7,8,284748,0.5733,0.4304,3.3157
8,9,275987,0.5557,0.4242,3.2577
9,10,268218,0.5400,0.4196,3.2176


In [7]:
mahala_afr_ratio = mahala_data[mahala_data['Count'] >= 6]
local_abnormal = pd.DataFrame()
for i in pd.date_range(range_min,range_max):
    df = mahala_afr_ratio[mahala_afr_ratio['NGAY_GD'] == i].nlargest(100, columns = 'Mahala_distance')
    local_abnormal = local_abnormal.append(df)
local_abnormal.head(10)

,NGAY_GD,CIF,Max_amt,Count,Mean,Std,Mahala_distance
276480,2021-12-01,03949893,150000.0,9,100000.000000,0.000000,inf
341888,2021-12-01,05213706,1000000.0,6,500000.000000,0.000000,inf
184171,2021-12-01,03325684,3300000.0,14,55128.571429,29418.215931,110.301435
297959,2021-12-01,04114327,1255000.0,8,21500.000000,11501.552690,107.246390
308327,2021-12-01,04192385,1500000.0,8,56250.000000,17677.669530,81.670833
129066,2021-12-01,02644110,5000000.0,8,79335.000000,60625.089337,81.165489
315518,2021-12-01,04238506,4000000.0,22,60920.136364,49726.215233,79.215356
243466,2021-12-01,03744510,5000000.0,18,156726.555556,70980.354661,68.233999
354890,2021-12-01,09476805,5000000.0,16,116562.500000,75666.565712,64.538908
110971,2021-12-01,02407752,4000000.0,110,55726.363636,67837.299525,58.143140


In [8]:
# wslib.save_data('local_abnormal.csv', local_abnormal.to_csv(index=False).encode())

{'name': 'local_abnormal.csv',
 'asset_type': 'data_asset',
 'asset_id': '967da601-ba2a-48cb-9b69-ffd4f75e7180',
 'attachment_id': 'f8445bc7-7fa8-44c5-83cf-0d5786b10327',
 'filepath': 'local_abnormal.csv',
 'data_size': None,
 'mime': 'text/csv',
 'summary': ['created file', 'created data asset', 'created attachment']}

# Truong hop Mahanobis_distance = Inf

In [16]:
local_inf = local_abnormal[local_abnormal['Std'] == 0]
local_inf['MaxCurrent/Mean'] = local_inf['Max_amt']/local_inf['Mean']
local_inf.quantile(np.linspace(.1,1,9,0))

/tmp/1000700000/ipykernel_1308/2257370442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  local_inf['MaxCurrent/Mean'] = local_inf['Max_amt']/local_inf['Mean']
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/numpy/lib/function_base.py:3961: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,Max_amt,Count,Mean,Std,Mahala_distance,MaxCurrent/Mean
0.1,40000.0,6.0,10000.0,0.0,NaN,1.86
0.2,82000.0,6.2,10000.0,0.0,NaN,2.00
0.3,190000.0,7.0,20000.0,0.0,NaN,2.00
0.4,216000.0,7.0,100000.0,0.0,NaN,2.00
0.5,300000.0,9.0,100000.0,0.0,NaN,2.50
0.6,560000.0,9.0,100000.0,0.0,NaN,3.00
0.7,1000000.0,9.0,200000.0,0.0,NaN,4.20
0.8,2842400.0,11.6,500000.0,0.0,NaN,5.00
0.9,9200000.0,14.4,5000000.0,0.0,NaN,12.00


# Thong tin chan dung

In [3]:
vip_data = """
SELECT LPAD(IDV_CST_DIM_ID,8,'0') AS CIF
	, CST_CGY_CODE  
FROM DMT.IDV_CST_DIM icd
"""
vip_data = pd.read_sql(vip_data, hdbi)
vip_data.head()

,CIF,CST_CGY_CODE
0,00431748,CA NHAN
1,00435625,CA NHAN
2,02985256,CA NHAN
3,30527765,CA NHAN
4,00424512,CA NHAN


In [4]:
casa_out = """
SELECT LEFT(AC_NO, 8) AS SO_CIF
	, SUM(AMT_LCY) AS KLGD_OUT
	, AVG(AMT_LCY) AS KLGD_TB_OUT
	, COUNT(TRN_REF_NO) AS SLGD_OUT
FROM CSO.TTDATA_ST_TXN tst 
WHERE year(PPN_DT) = '2021'
    AND MONTH (PPN_DT) = '12'
    AND TXN_DRC_F = 'D'
    AND AMT_LCY > 0
GROUP BY LEFT(AC_NO, 8)
"""
casa_out = pd.read_sql(casa_out, hdbi)
casa_out.head()

,SO_CIF,KLGD_OUT,KLGD_TB_OUT,SLGD_OUT
0,01449824,25809210.0000,3687030.0000,0.2258
1,03539623,5862000.0000,1954000.0000,0.0968
2,65576274,50000.0000,50000.0000,0.0323
3,00662269,13215000.0000,1651875.0000,0.2581
4,00838829,3279965.0000,655993.0000,0.1613


In [5]:
casa_in = """
SELECT LEFT(AC_NO, 8) AS SO_CIF
	, SUM(AMT_LCY) AS KLGD_IN
	, AVG(AMT_LCY) AS KLGD_TB_IN
	, COUNT(TRN_REF_NO) AS SLGD_IN
FROM CSO.TTDATA_ST_TXN tst 
WHERE year(PPN_DT) = '2021'
    AND MONTH (PPN_DT) = '12'
    AND TXN_DRC_F = 'C'
    AND AMT_LCY > 0
GROUP BY LEFT(AC_NO, 8)
"""
casa_in = pd.read_sql(casa_in, hdbi)
casa_in.head()

,SO_CIF,KLGD_IN,KLGD_TB_IN,SLGD_IN
0,02874175,24746088.0000,1649739.2000,0.4839
1,01089724,500000.0000,500000.0000,0.0323
2,04237241,1658974.0000,552991.3333,0.0968
3,01271753,29.0000,29.0000,0.0323
4,03326096,2.0000,2.0000,0.0323


In [6]:
casa = casa_out.merge(casa_in, on='SO_CIF', how='outer')

In [7]:
local_anomaly = pd.read_csv(wslib.mount.get_data_path('local_abnormal.csv'))
local_anomaly['CIF'] = local_anomaly['CIF'].astype('str').str.zfill(8)
local_anomaly.head()

,NGAY_GD,CIF,Max_amt,Count,Mean,Std,Mahala_distance
0,2021-12-01,03949893,150000.0000,9,100000.0000,0.0000,inf
1,2021-12-01,05213706,1000000.0000,6,500000.0000,0.0000,inf
2,2021-12-01,03325684,3300000.0000,14,55128.5714,29418.2159,110.3014
3,2021-12-01,04114327,1255000.0000,8,21500.0000,11501.5527,107.2464
4,2021-12-01,04192385,1500000.0000,8,56250.0000,17677.6695,81.6708


In [8]:
global_amount = pd.read_csv(wslib.mount.get_data_path('global_amount.csv'))
global_amount['CIF'] = global_amount['CIF'].astype('str').str.zfill(8)
global_amount.head()

,NGAY_GD,CIF,sum,max,min,mean,count
0,2021-12-01,02474414,30103000.0000,5000000.0000,50000.0000,3344777.7778,9
1,2021-12-01,04006352,25000000.0000,5000000.0000,5000000.0000,5000000.0000,5
2,2021-12-01,03449466,24125800.0000,5000000.0000,10000.0000,2010483.3333,12
3,2021-12-01,01994343,20363000.0000,5000000.0000,363000.0000,4072600.0000,5
4,2021-12-01,09355519,20200000.0000,5000000.0000,200000.0000,4040000.0000,5


In [9]:
global_count = pd.read_csv(wslib.mount.get_data_path('global_count.csv'))
global_count['CIF'] = global_count['CIF'].astype('str').str.zfill(8)
global_count.head()

,NGAY_GD,CIF,sum,max,min,mean,count
0,2021-12-01,08786107,555001.0000,50000.0000,10000.0000,27750.0500,20
1,2021-12-01,05664478,344256.0000,30000.0000,10000.0000,18118.7368,19
2,2021-12-01,03272149,1640000.0000,200000.0000,50000.0000,91111.1111,18
3,2021-12-01,03928410,624000.0000,230000.0000,10000.0000,34666.6667,18
4,2021-12-01,03934694,13760000.0000,2000000.0000,60000.0000,764444.4444,18


In [10]:
local_vip = local_anomaly.merge(vip_data, on = 'CIF', how = 'left')
local_anomaly_list = local_vip.merge(casa, left_on='CIF', right_on = 'SO_CIF', how = 'left')
local_anomaly_list

,NGAY_GD,CIF,Max_amt,Count,Mean,Std,Mahala_distance,CST_CGY_CODE,SO_CIF,KLGD_OUT,KLGD_TB_OUT,SLGD_OUT,KLGD_IN,KLGD_TB_IN,SLGD_IN
0,2021-12-01,03949893,150000.0000,9,100000.0000,0.0000,inf,CA NHAN,03949893,67889800.0000,1996758.8235,1.0968,66664953.0000,5128073.3077,0.4194
1,2021-12-01,05213706,1000000.0000,6,500000.0000,0.0000,inf,CA NHAN,05213706,13000000.0000,500000.0000,0.8387,13020003.0000,930000.2143,0.4516
2,2021-12-01,03325684,3300000.0000,14,55128.5714,29418.2159,110.3014,CA NHAN,03325684,36415812.0000,226185.1677,5.1935,36261202.0000,1066505.9412,1.0968
3,2021-12-01,04114327,1255000.0000,8,21500.0000,11501.5527,107.2464,CA NHAN,04114327,10076300.0000,167938.3333,1.9355,9827334.0000,204736.1250,1.5484
4,2021-12-01,04192385,1500000.0000,8,56250.0000,17677.6695,81.6708,CA NHAN,04192385,3011000.0000,752750.0000,0.1290,6000146.0000,1200029.2000,0.1613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,2021-12-31,02032886,5000000.0000,619,152117.4152,183993.2213,26.3482,CA NHAN,02032886,122519330.0000,2917126.9048,1.3548,123273786.0000,12327378.6000,0.3226
3096,2021-12-31,09851782,5000000.0000,10,231705.5000,181790.3129,26.2296,NaN,09851782,42562400.0000,1013390.4762,1.3548,33286610.0000,3026055.4545,0.3548
3097,2021-12-31,04342082,2640939.0000,28,80594.6071,97883.1448,26.1572,CA NHAN,04342082,10812421.0000,318012.3824,1.0968,11011942.0000,917661.8333,0.3871
3098,2021-12-31,02396154,750000.0000,61,34591.8033,27494.7952,26.0198,CA NHAN,02396154,27335700.0000,337477.7778,2.6129,25207133.0000,1145778.7727,0.7097


In [11]:
global_amount_vip = global_amount.merge(vip_data, on = 'CIF', how = 'left')
global_amount_list = global_amount_vip.merge(casa, left_on='CIF', right_on = 'SO_CIF', how = 'left')
global_amount_list

,NGAY_GD,CIF,sum,max,min,mean,count,CST_CGY_CODE,SO_CIF,KLGD_OUT,KLGD_TB_OUT,SLGD_OUT,KLGD_IN,KLGD_TB_IN,SLGD_IN
0,2021-12-01,02474414,30103000.0000,5000000.0000,50000.0000,3344777.7778,9,CA NHAN,02474414,1143457776.0000,4519595.9526,8.1613,1140485224.0000,8840970.7287,4.1613
1,2021-12-01,04006352,25000000.0000,5000000.0000,5000000.0000,5000000.0000,5,CA NHAN,04006352,167209052.0000,3981167.9048,1.3548,127240400.0000,6696863.1579,0.6129
2,2021-12-01,03449466,24125800.0000,5000000.0000,10000.0000,2010483.3333,12,CA NHAN,03449466,40268691.0000,529851.1974,2.4516,18545182.0000,842962.8182,0.7097
3,2021-12-01,01994343,20363000.0000,5000000.0000,363000.0000,4072600.0000,5,CA NHAN,01994343,101582141.0000,1846948.0182,1.7742,101154256.0000,4398011.1304,0.7419
4,2021-12-01,09355519,20200000.0000,5000000.0000,200000.0000,4040000.0000,5,CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,2021-12-31,02646307,15631000.0000,5000000.0000,631000.0000,3907750.0000,4,CA NHAN,02646307,80749219.0000,1468167.6182,1.7742,72355249.0000,7235524.9000,0.3226
3096,2021-12-31,03544179,15621830.0000,5000000.0000,621830.0000,3905457.5000,4,CF,03544179,404711222.0000,6323612.8438,2.0645,402107588.0000,2773155.7793,4.6774
3097,2021-12-31,03670522,15600000.0000,5000000.0000,600000.0000,3900000.0000,4,CA NHAN,03670522,122125540.0000,3131424.1026,1.2581,118534242.0000,5926712.1000,0.6452
3098,2021-12-31,02263515,15506000.0000,10000000.0000,357000.0000,3876500.0000,4,CA NHAN,02263515,212279062.0000,1739992.3115,3.9355,214002687.0000,2779255.6753,2.4839


In [12]:
global_count_vip = global_count.merge(vip_data, on = 'CIF', how = 'left')
global_count_list = global_count_vip.merge(casa, left_on='CIF', right_on = 'SO_CIF', how = 'left')
global_count_list

,NGAY_GD,CIF,sum,max,min,mean,count,CST_CGY_CODE,SO_CIF,KLGD_OUT,KLGD_TB_OUT,SLGD_OUT,KLGD_IN,KLGD_TB_IN,SLGD_IN
0,2021-12-01,08786107,555001.0000,50000.0000,10000.0000,27750.0500,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-12-01,05664478,344256.0000,30000.0000,10000.0000,18118.7368,19,CA NHAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-01,03272149,1640000.0000,200000.0000,50000.0000,91111.1111,18,CA NHAN,03272149,26651573.0000,233785.7281,3.6774,26490018.0000,441500.3000,1.9355
3,2021-12-01,03928410,624000.0000,230000.0000,10000.0000,34666.6667,18,CA NHAN,03928410,141725440.0000,745923.3684,6.1290,153436600.0000,2841418.5185,1.7419
4,2021-12-01,03934694,13760000.0000,2000000.0000,60000.0000,764444.4444,18,CA NHAN,03934694,375318034.0000,1022664.9428,11.8387,374900035.0000,3047967.7642,3.9677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,2021-12-31,04256805,9811400.0000,5000000.0000,74000.0000,981140.0000,10,CA NHAN,04256805,83968700.0000,711599.1525,3.8065,84329211.0000,1297372.4769,2.0968
3096,2021-12-31,04320567,742000.0000,100000.0000,40000.0000,74200.0000,10,CA NHAN,04320567,19902000.0000,133570.4698,4.8065,19902034.0000,265360.4533,2.4194
3097,2021-12-31,04321873,1200000.0000,200000.0000,100000.0000,120000.0000,10,CA NHAN,04321873,34079800.0000,202855.9524,5.4194,37400084.0000,526761.7465,2.2903
3098,2021-12-31,04384260,1630000.0000,200000.0000,100000.0000,163000.0000,10,NVFPTG,04384260,119066200.0000,452723.1939,8.4839,119066200.0000,1804033.3333,2.1290


In [13]:
wslib.save_data('local_anomaly_list.csv', local_anomaly_list.to_csv(index=False).encode())
wslib.save_data('global_amount_list.csv', global_amount_list.to_csv(index=False).encode())
wslib.save_data('global_count_list.csv', global_count_list.to_csv(index=False).encode())

{'name': 'global_count_list.csv',
 'asset_type': 'data_asset',
 'asset_id': '417720c7-b456-483c-a53a-899b05967102',
 'attachment_id': '739498dc-6ba1-4a3c-8471-83985eb8df54',
 'filepath': 'global_count_list.csv',
 'data_size': None,
 'mime': 'text/csv',
 'summary': ['created file', 'created data asset', 'created attachment']}